In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from utils.librascal import RascalSphericalExpansion

import ase.io

In [3]:
frames = ase.io.read("data/molecule_conformers_dftb.xyz", ":2")

In [4]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 6,
    "max_angular": 6,
    "gaussian_sigma_type": "Constant",
    "compute_gradients": True,
}

calculator = RascalSphericalExpansion(rascal_hypers)
descriptor = calculator.compute(frames)

/Users/guillaume/code/equistore-examples/utils/librascal.py:92: UserWarning: numpy/forward gradients are currently broken with librascal,please use rascaline instead
  warnings.warn(


In [5]:
# A TensorMap contains a set of keys, labeling the different blocks in the data

print(descriptor.keys.names)
print(descriptor.keys[:30:5])

('spherical_harmonics_l', 'center_species', 'neighbor_species')
[(0, 1, 1) (0, 6, 8) (1, 1, 6) (1, 8, 1) (2, 1, 8) (2, 8, 6)]


In [6]:
# we can also represent labels as named tuples or dictionaries
for i, label in enumerate(descriptor.keys.as_namedtuples()):
    if i > 3:
        break
    print(label)
    print(label.as_dict())
    print()

LabelTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=1)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 1}

LabelTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=6)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 6}

LabelTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=8)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 8}

LabelTuple(spherical_harmonics_l=0, center_species=6, neighbor_species=1)
{'spherical_harmonics_l': 0, 'center_species': 6, 'neighbor_species': 1}



In [7]:
# These labels can then be used to access different blocks
block = descriptor.block(
    spherical_harmonics_l=5, 
    center_species=1, 
    neighbor_species=1,
)

# a block contains a `values` array, the shape of which is determined by 
# three other set of labels: samples, components, and features
print(block.values.shape)

(12, 11, 6)


In [8]:
# The samples labels indicate **what** we are representing
print(block.samples.names)
print(block.samples[:6])

('structure', 'center')
[(0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9)]


In [9]:
# The components labels are optional and indicate components of tensorial/vector quantities
print(block.components[0].names)
print(block.components[0])

('spherical_harmonics_m',)
[(-5,) (-4,) (-3,) (-2,) (-1,) ( 0,) ( 1,) ( 2,) ( 3,) ( 4,) ( 5,)]


In [10]:
# The property labels indicate **how** we are representing the samples, here we
# are using a radial basis indexed by `n`
print(block.properties.names)
print(block.properties)

('n',)
[(0,) (1,) (2,) (3,) (4,) (5,)]


In [11]:
# the block can also contain gradients of the values w.r.t. different variables, 
# the most commong being the positions of the atoms in the system
gradient = block.gradient("positions")

# the gradients have their own set of sample and component labels, while the
# feature labels are shared with the values
print(gradient.data.shape)

print("\n samples:")
# the gradients samples indicate which value `sample` is being considered, and 
# with respect to which atom/spatial direction the gradients are being taken 
print(gradient.samples.names)
print(gradient.samples[:10])

# gradient with respect to position contain extra component labels
# indicating the direction of the gradient
print("\nfirst component:")
print(gradient.components[0].names)
print(gradient.components[0])

print("\nsecond component:")
print(gradient.components[1].names)
print(gradient.components[1])

(52, 3, 11, 6)

 samples:
('sample', 'structure', 'atom')
[(0, 0, 4) (0, 0, 9) (0, 0, 6) (0, 0, 5) (1, 0, 5) (1, 0, 9) (1, 0, 7)
 (1, 0, 4) (1, 0, 6) (2, 0, 6)]

first component:
('gradient_direction',)
[(0,) (1,) (2,)]

second component:
('spherical_harmonics_m',)
[(-5,) (-4,) (-3,) (-2,) (-1,) ( 0,) ( 1,) ( 2,) ( 3,) ( 4,) ( 5,)]


In [12]:
# since there is a single oxygen atom, there are no contribution to the gradient
# with center_specie=8, neighbor_species=8, spherical_harmonics_l>0
block = descriptor.block(
    spherical_harmonics_l=4, 
    center_species=8, 
    neighbor_species=8,
)

gradients = block.gradient("positions")
assert len(gradients.samples) == 0

# Moving labels around (from sparse to dense storage)

In [13]:
rascal_hypers["compute_gradients"] = False

calculator = RascalSphericalExpansion(rascal_hypers)
descriptor = calculator.compute(frames)

In [14]:
# we can group together multiple block by moving a label from keys to the propertie

descriptor.keys_to_properties("neighbor_species")

In [15]:
block = descriptor.block(center_species=1, spherical_harmonics_l=6)

print(block.values.shape)
print(block.properties.names)
print(block.samples)

(12, 13, 18)
('neighbor_species', 'n')
[(0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9) (1, 4) (1, 5) (1, 6) (1, 7)
 (1, 8) (1, 9)]


In [16]:
# depending on the blocks, we might not be able to move all sparse labels to
# features
try:
    descriptor.keys_to_properties("spherical_harmonics_l")
except Exception as e:
    print(e)

invalid parameter: can not move keys to properties if the blocks have different components labels, call components_to_properties first


In [17]:
# we need to move the m index to features before moving l to features
descriptor.components_to_properties("spherical_harmonics_m")

descriptor.keys_to_properties("spherical_harmonics_l")

In [18]:
block = descriptor.block(center_species=1)
block.values.shape

(12, 882)

In [19]:
descriptor.keys_to_samples("center_species")

In [20]:
# we now only have one block, containing everything
block = descriptor.block()
block.values.shape

block.samples

Labels([(0, 0, 8), (0, 1, 6), (0, 2, 6), (0, 3, 6), (0, 4, 1), (0, 5, 1),
        (0, 6, 1), (0, 7, 1), (0, 8, 1), (0, 9, 1), (1, 0, 8), (1, 1, 6),
        (1, 2, 6), (1, 3, 6), (1, 4, 1), (1, 5, 1), (1, 6, 1), (1, 7, 1),
        (1, 8, 1), (1, 9, 1)],
       dtype=[('structure', '<i4'), ('center', '<i4'), ('center_species', '<i4')])

## Checking vs librascal storage

In [21]:
calculator = RascalSphericalExpansion(rascal_hypers)
descriptor = calculator.compute(frames)

descriptor.keys_to_properties("neighbor_species")
species_radial_size = descriptor.block(spherical_harmonics_l=0, center_species=1).values.shape[2]

descriptor.components_to_properties("spherical_harmonics_m")
descriptor.keys_to_properties("spherical_harmonics_l")
descriptor.keys_to_samples("center_species")

block = descriptor.block()
full_dense = block.values

# put lm at the end of the features
full_dense = full_dense.reshape(full_dense.shape[0], -1, species_radial_size)
full_dense = full_dense.swapaxes(1, 2)
full_dense = full_dense.reshape(full_dense.shape[0], -1)

In [22]:
from rascal.representations import SphericalExpansion

rascal_calculator = SphericalExpansion(**rascal_hypers)
managers = rascal_calculator.transform(frames)
rascal_spx = managers.get_features(rascal_calculator)

In [23]:
assert np.all(rascal_spx == full_dense)